In [3]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

# --- 1. Memuat data referensi dari Nama_Orang.xlsx ---
df_names = pd.read_csv('train_data.csv')

df_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256023 entries, 0 to 256022
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   NO_PERMOHONAN          256023 non-null  object 
 1   FILE_SER               256023 non-null  int64  
 2   T1_SERVICE_PERSON_NBR  256023 non-null  object 
 3   USER_ID                59972 non-null   float64
 4   USER_NAME              59972 non-null   object 
 5   USER_TYPE_NAME         59952 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 11.7+ MB


C:\Users\FX506\AppData\Local\Temp\ipykernel_27208\4023131581.py:6: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_names = pd.read_csv('train_data.csv')


In [ ]:
# Membuat daftar nama orang untuk fuzzy matching
official_names = list({row['Nama_konsultan'].strip().upper()
                       for _, row in df_names.iterrows()
                       if pd.notna(row['Nama_konsultan']) and row['Nama_konsultan'].strip() != ''})

# --- 2. Definisikan fungsi pembersihan teks ---
def clean_text(text, official_names, fuzzy_threshold=85):
    text = text.upper().strip()
    print("\nMemproses:", text)
    
    # --- Fuzzy matching untuk mencocokkan nama ---
    best_match, score, _ = process.extractOne(text, official_names, scorer=fuzz.token_sort_ratio)
    if score >= fuzzy_threshold:
        text = best_match
        print("Fuzzy matching sukses: '{}' -> '{}' (skor {})".format(text, best_match, score))
    
    text = ' '.join(text.split())
    print("Hasil akhir:", text)
    return text

# --- 3. Memuat data uji dan menerapkan fungsi cleansing ---
df_uji = pd.read_csv('data_uji.csv')
df_uji['cleaned_text'] = df_uji['Nama Konsultan'].apply(
    lambda x: clean_text(x, official_names)
)

# --- 4. Simpan hasil ---
df_uji.to_csv('clean_data.csv', index=False)
print("\nData cleansing selesai. Hasil disimpan ke 'clean_data.csv'.")


Memproses: A A DANI SALISWIJAYA S.H., M.H.
Hasil akhir: A A DANI SALISWIJAYA S.H., M.H.

Memproses: ABDUL KARIM S.E., S.H.
Fuzzy matching sukses: 'ABDUL KARIM S.E., S.H.' -> 'ABDUL KARIM S.E., S.H.' (skor 100.0)
Hasil akhir: ABDUL KARIM S.E., S.H.

Memproses: ABDULLAH LOETFI (ALM) S.H.
Fuzzy matching sukses: 'ABDULLAH LOETFI, S.H.' -> 'ABDULLAH LOETFI, S.H.' (skor 85.1063829787234)
Hasil akhir: ABDULLAH LOETFI, S.H.

Memproses: ACHMAD FATCHY, S.H
Fuzzy matching sukses: 'ACHMAD FATCHY, S.H' -> 'ACHMAD FATCHY, S.H' (skor 100.0)
Hasil akhir: ACHMAD FATCHY, S.H

Memproses: DRS. ACHMAD YANI
Fuzzy matching sukses: 'DRS. ACHMAD YANI' -> 'DRS. ACHMAD YANI' (skor 100.0)
Hasil akhir: DRS. ACHMAD YANI

Memproses: DR. H. ADI TAGOR HARAHAP SP.A, D.P.H.
Hasil akhir: DR. H. ADI TAGOR HARAHAP SP.A, D.P.H.

Memproses: ADOLF MARTUA PANGGABEAN S.H.
Fuzzy matching sukses: 'ADOLF MARTUA PANGGABEAN, S.H.' -> 'ADOLF MARTUA PANGGABEAN, S.H.' (skor 98.24561403508771)
Hasil akhir: ADOLF MARTUA PANGGABEAN, S.H.

In [7]:
df_uji.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101 entries, 0 to 1100
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Nama Konsultan        1101 non-null   object
 1   No Konsultan          1101 non-null   object
 2   Alamat Korespondensi  1101 non-null   object
 3   Alamat Email          1043 non-null   object
 4   No Telp               995 non-null    object
 5   Status                1101 non-null   object
 6   cleaned_text          1101 non-null   object
dtypes: object(7)
memory usage: 60.3+ KB
